In [76]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

# List of PyTorch optimizers
optimizers = [
    optim.SGD,               # Stochastic Gradient Descent
    optim.Adam,              # Adam optimizer
    optim.AdamW,             # AdamW optimizer
    optim.Adagrad,           # Adagrad optimizer
    optim.Adadelta,          # Adadelta optimizer
    optim.RMSprop,           # RMSprop optimizer
    optim.Adamax,            # Adamax optimizer
    optim.ASGD,              # Averaged Stochastic Gradient Descent
    optim.LBFGS,             # Limited-memory BFGS
    optim.Rprop,             # Resilient backpropagation
    optim.SparseAdam,        # Sparse Adam optimizer
    optim.NAdam,             # Nesterov-accelerated Adaptive Moment Estimation
    optim.RAdam,             # Rectified Adam optimizer
]
def train_model(model, train_data, train_labels, epochs=10, learning_rate=0.001, optim_index=1):
    # Set the model to training mode
    model.train()
    train_labels = train_labels.view(-1, 1)
    # Define loss function and optimizer
    criterion = torch.nn.BCELoss()  # Use BCE for binary output (CrossEntropy if multiple classes)
    optimizer = optimizers[optim_index](model.parameters(), lr=learning_rate)

    # Convert data to PyTorch tensors
    dataset = TensorDataset(train_data, train_labels)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Training loop
    for epoch in range(epochs):
        epoch_loss = 0.0
        for inputs, labels in dataloader:
            # Zero gradients from the previous step
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()

            # Update weights
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

import torch
import torch.nn as nn
import numpy as np

class FFN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFN, self).__init__()
        layers = []
        sizes = [input_size] + hidden_sizes

        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes) -  1:  # No activation for the final layer
                layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-1], 1))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def get_parameters(self):
        """Retrieve all model parameters as a list of tensors."""
        return [p.clone().detach() for p in self.parameters()]

    def set_parameters(self, parameters):
        """Set model parameters from a list of tensors."""
        with torch.no_grad():
            for param, new_param in zip(self.parameters(), parameters):
                param.copy_(new_param)
    def extract_features(self, x):
        """Get the feature representation from the last hidden layer"""
        _, features = self.forward(x, return_features=True)
        return features[-1]  # Return the last hidden layer activation

# Create an initial population of 20 FFNs
population = [FFN(98, [20, 15], 2) for _ in range(20)]


In [70]:
def select_dataset(dataset_name):
  fairness_to_select = None
  if dataset_name == 'compas':

    import pandas as pd
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline

    # URL to the COMPAS dataset
    url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"

    # Load the dataset
    df_compas = pd.read_csv(url)
    """
    # Display the first few rows of the dataset


    # Identify categorical columns
    categorical_columns = df_compas.select_dtypes(include=['object', 'category']).columns.tolist()

    # Initialize the LabelEncoder and OneHotEncoder
    label_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)

    # Apply Label Encoding to binary categorical columns
    binary_categorical_cols = ['sex', 'score_text']  # Example binary columns
    for col in binary_categorical_cols:
        if col in categorical_columns:
            df_compas[col] = label_encoder.fit_transform(df_compas[col])

    # Apply One-Hot Encoding to other categorical columns
    other_categorical_cols = [col for col in categorical_columns if col not in binary_categorical_cols]

    # Use ColumnTransformer to apply one-hot encoding
    preprocessor = ColumnTransformer(
        transformers=[
            ('onehot', one_hot_encoder, other_categorical_cols)
        ],
        remainder='passthrough'
    )

    # Apply the transformations
    df_compas_encoded = preprocessor.fit_transform(df_compas)

    # Convert the result to a DataFrame
    df_compas_encoded = pd.DataFrame(df_compas_encoded, columns=preprocessor.get_feature_names_out())

    # Display the first few rows of the encoded dataset


    """
    df_compas = df_compas.drop('id',  axis=1)
    df_compas = df_compas.drop('dob',  axis=1)
    df_compas = df_compas.drop('compas_screening_date',  axis=1)
    df_compas = df_compas.drop('c_case_number',  axis=1)
    df_compas = df_compas.drop('screening_date', axis=1)
    df_compas = df_compas.drop('in_custody', axis=1)
    df_compas = df_compas.drop('out_custody', axis=1)
    df_compas = df_compas.drop('v_screening_date', axis=1)
    df_compas = df_compas.drop('c_offense_date', axis=1)
    df_compas = df_compas.drop('c_jail_out', axis=1)
    df_compas = df_compas.drop('c_jail_in', axis=1)
    from sklearn.preprocessing import LabelEncoder

    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()

    # Apply Label Encoding to a specific column
    df_compas['name'] = label_encoder.fit_transform(df_compas['name'])
    df_compas['first'] = label_encoder.fit_transform(df_compas['first'])
    df_compas['last'] = label_encoder.fit_transform(df_compas['last'])
    df_compas['sex'] = label_encoder.fit_transform(df_compas['sex'])
    df_compas['score_text'] = label_encoder.fit_transform(df_compas['score_text'])
    df_compas['v_type_of_assessment'] = label_encoder.fit_transform(df_compas['v_type_of_assessment'])
    df_compas['c_charge_desc'] = label_encoder.fit_transform(df_compas['c_charge_desc'])

    df_compas['r_case_number'] = label_encoder.fit_transform(df_compas['r_case_number'])
    df_compas['c_arrest_date'] = label_encoder.fit_transform(df_compas['c_arrest_date'])
    df_compas['r_charge_degree'] = label_encoder.fit_transform(df_compas['r_charge_degree'])
    df_compas['vr_case_number'] = label_encoder.fit_transform(df_compas['vr_case_number'])
    df_compas['vr_charge_degree'] = label_encoder.fit_transform(df_compas['vr_charge_degree'])
    df_compas['vr_offense_date'] = label_encoder.fit_transform(df_compas['vr_offense_date'])
    df_compas['vr_charge_desc'] = label_encoder.fit_transform(df_compas['vr_charge_desc'])
    df_compas['r_offense_date'] = label_encoder.fit_transform(df_compas['r_offense_date'])
    df_compas['r_charge_desc'] = label_encoder.fit_transform(df_compas['r_charge_desc'])
    df_compas['r_jail_in'] = label_encoder.fit_transform(df_compas['r_jail_in'])
    df_compas['r_jail_out'] = label_encoder.fit_transform(df_compas['r_jail_out'])

    import pandas as pd
    df_compas_encoded = pd.get_dummies(df_compas, columns = ['age_cat', 'race', 'c_charge_degree', 'type_of_assessment', 'v_score_text'])

    df_compas_encoded = df_compas_encoded.fillna(0)

    import torch
    import torch.nn as nn
    import numpy as np
    from sklearn.model_selection import train_test_split
    X = df_compas_encoded.drop(['two_year_recid'], axis=1)
    y = df_compas_encoded['two_year_recid']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #print(X_test.columns)
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(np.vstack(X_train.values).astype(np.float64), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(np.vstack(X_test.values).astype(np.float64), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
    input_size = X_train_tensor.shape[1]
    output_size = 2
    fairness_to_select = 1
  else :

    from sklearn.datasets import fetch_openml
    from sklearn.preprocessing import LabelEncoder

    import numpy as np
    import pandas as pd

    # Fetch UCI Adult Income Dataset
    data = fetch_openml("adult", version=2, as_frame=True)
    df = data.frame
    #print((df.columns))

    # Encode target labels ('<=50K' or '>50K')
    label_encoder = LabelEncoder()
    df['income'] = label_encoder.fit_transform(df['class'])
    #print((df['income']))

    import torch
    import torch.nn as nn
    import numpy as np
    from sklearn.model_selection import train_test_split
    # Convert boolean columns to integers (0 for False, 1 for True)
    df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

    # The rest remains unchanged

    df = pd.get_dummies(df, drop_first=True)
    df = df.fillna(0)

    X = df.drop(['income', 'class_>50K'], axis=1)
    y = df['income']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(np.vstack(X_train.values).astype(np.float32), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(np.vstack(X_test.values).astype(np.float32), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

    input_size = X_train_tensor.shape[1]
    output_size = 2
    fairness_to_select = 0
  return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, X_test, input_size, output_size, fairness_to_select

In [92]:
import torch
import torch.nn.functional as F


def fgsm_attack(model, data, target, epsilon):
    """Generates adversarial examples using FGSM."""
    data.requires_grad = True
    output = model(data)
    loss = F.binary_cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    return torch.clamp(perturbed_data, 0, 1)

def adversarial_train(model, X_train_tensor, y_train_tensor, optimizer, device, epsilon=0.1, attack_type='fgsm'):
    """Performs adversarial training using FGSM."""

    model.train()
    epochs = 10
    for epoch in range(epochs):
      epoch_loss = 0.0
      X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
      y_train_tensor = y_train_tensor.view(-1, 1)
      # Generate adversarial examples
      if attack_type == 'fgsm':
          adv_data = fgsm_attack(model, X_train_tensor, y_train_tensor, epsilon)
      else:
          adv_data = X_train_tensor  # Placeholder for PGD (can be implemented later)

      # Combine clean and adversarial examples
      mixed_data = torch.cat([X_train_tensor, adv_data])
      mixed_target = torch.cat([y_train_tensor, y_train_tensor])

      # Forward pass
      optimizer.zero_grad()
      output = model(mixed_data)
      loss = F.binary_cross_entropy(output, mixed_target)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()

      print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")


In [74]:
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,X_test, input_size, output_size, fairness_to_select = select_dataset('compas')
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)
sensetive_features = [['sex_Male', 'race_Black', 'race_White'], ['sex',  'race_African-American', 'race_Asian']]

In [68]:
model = FFN(input_size, [20, 15], 2)

In [69]:
train_model(model, X_train_tensor, y_train_tensor)

torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])
torch.Size([32, 1]) torch.Size([32, 1])


In [83]:
from sklearn.metrics import accuracy_score, confusion_matrix
def compute_class_features(model, X, y, num_classes):
    class_features = {i: [] for i in range(num_classes)}

    with torch.no_grad():
        for i in range(len(X)):
            features = model.extract_features(X[i].unsqueeze(0))
            class_features[y[i].item()].append(features)

    # Compute mean feature representation for each class
    for cls in class_features:
        if class_features[cls]:  # Avoid empty class
            class_features[cls] = torch.mean(torch.stack(class_features[cls]), dim=0)

    return class_features

def cafa_attack(model, X, y, target_class_features, epsilon=0.1):
    X_adv = X.clone().detach().requires_grad_(True)

    # Get original feature representation
    features = model.extract_features(X_adv)

    # Compute the feature shift direction towards target class
    target_features = target_class_features[y.item()]
    loss = torch.nn.functional.mse_loss(features, target_features)

    # Compute gradient and generate adversarial example
    loss.backward()
    X_adv = X_adv - epsilon * X_adv.grad.sign()
    return X_adv.detach()

def compute_accuracy(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        #_, predictions = torch.max(outputs, 1)

    #return accuracy_score(y_test.numpy(), predictions.numpy())
    return ((outputs > 0.5).float() == y_test).float().mean()

import torch
import torch.nn as nn

def pgd_adversarial_attack(model, X, y, epsilon=1, alpha=0.01, num_iter=40):
    """
    Projected Gradient Descent (PGD) attack.

    Args:
        model: The target model to attack.
        X: Input data (requires gradients).
        y: Correct labels for the input data.
        epsilon: Maximum allowed perturbation per input dimension.
        alpha: Step size for each iteration.
        num_iter: Number of iterations for the attack.

    Returns:
        Adversarial examples generated from X.
    """
    # Clone input and set it to require gradients
    X_adv = X.clone().detach().requires_grad_(True)
    loss_fn = nn.BCELoss()

    for _ in range(num_iter):
        # Zero the gradients
        model.zero_grad()

        # Forward pass and compute loss
        output = model(X_adv)
        loss = loss_fn(output, y)
        loss.backward()

        # Update adversarial examples with the gradient sign
        perturbation = alpha * X_adv.grad.sign()
        X_adv = X_adv + perturbation

        # Project back to the epsilon ball around the original inputs
        X_adv = torch.max(torch.min(X_adv, X + epsilon), X - epsilon)
        X_adv = torch.clamp(X_adv, 0, 1)  # Keep pixel values between 0 and 1

        # Reset gradient computation
        X_adv = X_adv.detach().requires_grad_(True)

    return X_adv

def fgsm_attack(model, X, y, epsilon):
    X.requires_grad_()  # Ensure gradients are computed
    model.zero_grad()  # Clear previous gradients
    output = model(X)
    loss = nn.BCELoss()(output, y)
    loss.backward()
    perturbation = epsilon * X.grad.sign()
    X_adv = torch.clamp(X + perturbation, 0, 1)
    return X_adv
def adversarial_attack_test_data(model, X, y, attack_name):
  if attack_name == 'PGD':
    X_adv = pgd_adversarial_attack(model, X, y)
  else:
    X_adv = fgsm_attack(model, X, y, 0.05)
  return X_adv
def adversarial_attack_accuracy(model, X, y, attack_name):
  if attack_name == 'PGD':
    X_adv = pgd_adversarial_attack(model, X, y)
    rslt = compute_accuracy(model, X_adv, y)
  else:
    X_adv = fgsm_attack(model, X, y, 0.05)
    rslt = compute_accuracy(model, X_adv, y)
  return rslt
from sklearn.metrics import confusion_matrix

def compute_tpr_fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # Compute TPR and FPR
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    return tpr, fpr
import torch
import torch.nn as nn
import numpy as np

def compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, sensitive_feature):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        predictions = (outputs > 0.5).float()  # Assuming binary classification with threshold 0.5

    # Convert sensitive_feature to torch tensor if it isn't already
    sensitive_feature_tensor = torch.tensor(sensitive_feature.values, dtype=torch.int32)

    # Calculate true positive rates (TPR) and false positive rates (FPR) for each group
    tpr_group_0 = ((predictions[sensitive_feature_tensor == 0] == 1) & (y_test_tensor[sensitive_feature_tensor == 0] == 1)).float().mean().item()
    tpr_group_1 = ((predictions[sensitive_feature_tensor == 1] == 1) & (y_test_tensor[sensitive_feature_tensor == 1] == 1)).float().mean().item()

    fpr_group_0 = ((predictions[sensitive_feature_tensor == 0] == 1) & (y_test_tensor[sensitive_feature_tensor == 0] == 0)).float().mean().item()
    fpr_group_1 = ((predictions[sensitive_feature_tensor == 1] == 1) & (y_test_tensor[sensitive_feature_tensor == 1] == 0)).float().mean().item()

    # Compute equalized odds difference
    tpr_difference = abs(tpr_group_0 - tpr_group_1)
    fpr_difference = abs(fpr_group_0 - fpr_group_1)

    equalized_odds_difference = max(tpr_difference, fpr_difference)

    return tpr_difference, fpr_difference

# Example usage
# Assuming you have a model, test data, and sensitive feature
# model = ...
# X_test_tensor = ...
# y_test_tensor = ...
# sensitive_feature = ...

# equalized_odds_diff = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, sensitive_feature)
# print("Equalized Odds Difference:", equalized_odds_diff)



In [78]:
model = FFN(input_size, [20, 15], 2)
adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.1, attack_type='pgd')
adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')



In [79]:
compute_accuracy(model, X_test_tensor, y_test_tensor)

tensor(0.5489)

In [80]:
adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')

tensor(0.4241)

In [81]:
adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')

tensor(0.2571)

In [72]:
!pip install pymoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.0 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=779bf329cdb1e1001424c26479154ffa0491f1fa30b39c4b8b92cc7f6348102e
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


In [93]:

import numpy as np
import torch
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.core.variable import Real
from pymoo.termination import get_termination

import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.callback import Callback
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

class FFNProblem(Problem):
    def __init__(self, max_layers, max_nodes, X_train, X_test, y_train, y_test):
        super().__init__(n_var=max_layers+2,
                         n_obj=5,
                         n_constr=0,
                         xl=5,
                         xu=max_nodes,
                         type_var=int)
        self.max_layers = max_layers
        self.max_nodes = max_nodes
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.count = 0
    def _evaluate(self, x, out, *args, **kwargs):
        adversarial_accuracy_PGD = np.zeros(x.shape[0])
        adversarial_accuracy_FGSM = np.zeros(x.shape[0])
        equalized_odds_gender = np.zeros(x.shape[0])
        equalized_odds_race_Black = np.zeros(x.shape[0])
        equalized_odds_race_White = np.zeros(x.shape[0])
        self.count += 1
        print(self.count)
        print(x)
        for i, architecture in enumerate(x):
            l, learning_rate_and_ind_opt = architecture[:-2], architecture[-2:]


            learning_rate = learning_rate_and_ind_opt[0]
            ind_opt = int(learning_rate_and_ind_opt[1]-4)
            lr = 10 ** (-1*float(learning_rate)+4)
            print(lr, ind_opt)
            l = np.round(l).astype(int)
            print(l)
            temp = l.tolist()
            print(input_size)
            model = FFN(input_size, temp, 2)

            #train_model(model, X_train_tensor, y_train_tensor, 5, lr, ind_opt)
            adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')
            #adv_acc, eq_odds = evaluate_model(architecture, self.X_train, self.X_test, self.y_train, self.y_test)
            accuracy_pgd = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')
            accuracy_fgsm = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
            tpr_diff_gender, fpr_diff_gender = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][0]])
            tpr_diff_race_Black, fpr_diff_race_Black = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][1]])
            tpr_diff_race_White, fpr_diff_race_White = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][2]])

            equalized_odd_gender = (tpr_diff_gender + fpr_diff_gender) / 2
            equalized_odd_race_Black = (tpr_diff_race_Black + fpr_diff_race_Black) / 2
            equalized_odd_race_White = (tpr_diff_race_White + fpr_diff_race_White) / 2

            adversarial_accuracy_PGD[i] = accuracy_pgd
            adversarial_accuracy_FGSM[i] = accuracy_fgsm

            equalized_odds_gender[i] = equalized_odd_gender
            equalized_odds_race_Black[i] = equalized_odd_race_Black
            equalized_odds_race_White[i] = equalized_odd_race_White

        out["F"] = np.column_stack([adversarial_accuracy_PGD, adversarial_accuracy_FGSM, equalized_odds_gender, equalized_odds_race_Black, equalized_odds_race_White])


# Initialize the problem
max_layers = 5  # Example: maximum number of layers
max_nodes = 10  # Example: maximum number of nodes per layer
problem = FFNProblem(max_layers, max_nodes, X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor)

# Define crossover and mutation operators
crossover = SBX(prob=0.9, eta=15)
mutation = PM(eta=20)

# Initialize the algorithm
#algorithm = NSGA2(pop_size=10, sampling=np.array(population))

initial_population = np.array([[7, 7, 7, 7, 6, 5, 6], [6, 9, 7, 7, 9, 8, 9], [6, 7, 6, 8, 6, 9, 7], [7, 7, 8, 7, 7, 7, 6]])

# Initialize the algorithm with the custom initial population
algorithm = NSGA2(
    pop_size=4,
    sampling = initial_population,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True
)

# Set the initial population



# Define the termination criterion
termination = get_termination("n_gen", 2)

# Run the optimization
res = minimize(problem,
               algorithm,
               termination,
               verbose=True)

# Print the results
print("Best solution found: %s" % res.F[0])
print("Best architecture found: %s" % res.X[0])

1
[[7 7 7 7 6 5 6]
 [6 9 7 7 9 8 9]
 [6 7 6 8 6 9 7]
 [7 7 8 7 7 7 6]]
0.1 2
[7 7 7 7 6]
51
Epoch 1/10, Loss: 0.7589
Epoch 2/10, Loss: 0.7473
Epoch 3/10, Loss: 0.7369
Epoch 4/10, Loss: 0.7276
Epoch 5/10, Loss: 0.7191
Epoch 6/10, Loss: 0.7112
Epoch 7/10, Loss: 0.7039
Epoch 8/10, Loss: 0.6971
Epoch 9/10, Loss: 0.6902
Epoch 10/10, Loss: 0.6840
0.0001 5
[6 9 7 7 9]
51
Epoch 1/10, Loss: 0.8236
Epoch 2/10, Loss: 0.8019
Epoch 3/10, Loss: 0.7821
Epoch 4/10, Loss: 0.7641
Epoch 5/10, Loss: 0.7478
Epoch 6/10, Loss: 0.7321
Epoch 7/10, Loss: 0.7172
Epoch 8/10, Loss: 0.7038
Epoch 9/10, Loss: 0.6917
Epoch 10/10, Loss: 0.6812
1e-05 3
[6 7 6 8 6]
51
Epoch 1/10, Loss: 1.0866
Epoch 2/10, Loss: 1.0180
Epoch 3/10, Loss: 0.9603
Epoch 4/10, Loss: 0.9126
Epoch 5/10, Loss: 0.8729
Epoch 6/10, Loss: 0.8394
Epoch 7/10, Loss: 0.8112
Epoch 8/10, Loss: 0.7874
Epoch 9/10, Loss: 0.7669
Epoch 10/10, Loss: 0.7493
0.001 2
[7 7 8 7 7]
51
Epoch 1/10, Loss: 0.7297
Epoch 2/10, Loss: 0.7246
Epoch 3/10, Loss: 0.7194
Epoch 4/10

In [85]:
def mutate(ffn, mutation_rate=0.1, mutation_strength=0.01):
    parameters = ffn.get_parameters()
    for i in range(len(parameters)):
        if np.random.rand() < mutation_rate:
            noise = torch.randn_like(parameters[i]) * mutation_strength
            parameters[i] += noise
    ffn.set_parameters(parameters)


In [86]:
population_size = 4
population_structures = []
for i in range(population_size):
    population_structures.append(res.X[0].tolist())



In [95]:
from copy import deepcopy
# Train 5 initial models on the COMPAS dataset
trained_models = []
for _ in population_structures:

    l = []
    for i in range(len(_)-2):
      l.append(int(_[i]))


    model = FFN(input_size, l, 2)
    adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')
    #train_model(model, X_train_tensor, y_train_tensor)  # Assume this function trains the model
    trained_models.append(model)

# Clone and mutate to complete the population

population = []
for _ in range(population_size):
    base_model = deepcopy(trained_models[np.random.randint(population_size)])
    mutate(base_model)  # Apply slight mutation for diversity
    population.append(base_model)


Epoch 1/10, Loss: 0.7643
Epoch 2/10, Loss: 0.7463
Epoch 3/10, Loss: 0.7316
Epoch 4/10, Loss: 0.7199
Epoch 5/10, Loss: 0.7102
Epoch 6/10, Loss: 0.7020
Epoch 7/10, Loss: 0.6957
Epoch 8/10, Loss: 0.6915
Epoch 9/10, Loss: 0.6892
Epoch 10/10, Loss: 0.6878
Epoch 1/10, Loss: 1.4093
Epoch 2/10, Loss: 1.1886
Epoch 3/10, Loss: 1.0192
Epoch 4/10, Loss: 0.8941
Epoch 5/10, Loss: 0.8085
Epoch 6/10, Loss: 0.7551
Epoch 7/10, Loss: 0.7237
Epoch 8/10, Loss: 0.7072
Epoch 9/10, Loss: 0.6983
Epoch 10/10, Loss: 0.6946
Epoch 1/10, Loss: 1.3687
Epoch 2/10, Loss: 1.1592
Epoch 3/10, Loss: 1.0068
Epoch 4/10, Loss: 0.9028
Epoch 5/10, Loss: 0.8322
Epoch 6/10, Loss: 0.7835
Epoch 7/10, Loss: 0.7498
Epoch 8/10, Loss: 0.7267
Epoch 9/10, Loss: 0.7117
Epoch 10/10, Loss: 0.7019
Epoch 1/10, Loss: 0.8254
Epoch 2/10, Loss: 0.7912
Epoch 3/10, Loss: 0.7661
Epoch 4/10, Loss: 0.7487
Epoch 5/10, Loss: 0.7361
Epoch 6/10, Loss: 0.7250
Epoch 7/10, Loss: 0.7134
Epoch 8/10, Loss: 0.7011
Epoch 9/10, Loss: 0.6891
Epoch 10/10, Loss: 0.6

In [52]:
sum(param.numel() for param in population[0].get_parameters())

809

In [53]:
population[0].parameters()

<generator object Module.parameters at 0x7e76bf3a4200>

In [54]:
for i, param in enumerate(population[0].get_parameters()):
  print(i)
  print(param[0].shape)

0
torch.Size([97])
1
torch.Size([])
2
torch.Size([6])
3
torch.Size([])
4
torch.Size([7])
5
torch.Size([])
6
torch.Size([6])
7
torch.Size([])
8
torch.Size([8])
9
torch.Size([])
10
torch.Size([6])
11
torch.Size([])


In [96]:
import numpy as np
import torch
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.callback import Callback


# Define the optimization problem
class FFNOptimizationProblem(Problem):
    def __init__(self, model):
        self.model = model


        # Flatten the parameters to create a single vector of variables
        self.original_params = model.get_parameters()
        self.param_shapes = [param.shape for param in self.original_params]
        self.num_vars = sum(param.numel() for param in self.original_params)

        super().__init__(n_var=self.num_vars, n_obj=5, n_constr=0, xl=-1.0, xu=1.0)

    def _evaluate(self, x, out, *args, **kwargs):
        adversarial_accuracy_PGD = np.zeros(x.shape[0])
        adversarial_accuracy_FGSM = np.zeros(x.shape[0])
        equalized_odds_gender = np.zeros(x.shape[0])
        equalized_odds_race_Black = np.zeros(x.shape[0])
        equalized_odds_race_White = np.zeros(x.shape[0])
        # Reshape the flat vector back into the model parameters
        for i in range(x.shape[0]):  # Iterate over each individual in the population
            params = self._reshape_params(x[i])
            self.model.set_parameters(params)
            model = self.model
            accuracy_pgd = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')
            accuracy_fgsm = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
            tpr_diff_gender, fpr_diff_gender = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][0]])
            tpr_diff_race_Black, fpr_diff_race_Black = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][1]])
            tpr_diff_race_White, fpr_diff_race_White = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][2]])

            equalized_odd_gender = (tpr_diff_gender + fpr_diff_gender) / 2
            equalized_odd_race_Black = (tpr_diff_race_Black + fpr_diff_race_Black) / 2
            equalized_odd_race_White = (tpr_diff_race_White + fpr_diff_race_White) / 2

            adversarial_accuracy_PGD[i] = accuracy_pgd
            adversarial_accuracy_FGSM[i] = accuracy_fgsm

            equalized_odds_gender[i] = equalized_odd_gender
            equalized_odds_race_Black[i] = equalized_odd_race_Black
            equalized_odds_race_White[i] = equalized_odd_race_White

        out["F"] = np.column_stack([adversarial_accuracy_PGD, adversarial_accuracy_FGSM, equalized_odds_gender, equalized_odds_race_Black, equalized_odds_race_White])





    def _reshape_params(self, x):
        params = []
        index = 0
        for shape in self.param_shapes:
            size = np.prod(shape)
            params.append(torch.tensor(x[index:index + size].reshape(shape), dtype=torch.float32))
            index += size
        return params


model = population[0]


# Load pretrained models (example)
pretrained_models = population
# Assume pretrained_models are already trained and parameters are set

# Flatten parameters of pretrained models to use as initial population
initial_population = []
for pretrained_model in pretrained_models:
    params = pretrained_model.get_parameters()
    flat_params = np.concatenate([p.numpy().flatten() for p in params])
    initial_population.append(flat_params)

# Convert initial population to a numpy array
initial_population = np.array(initial_population)

# Define the problem
problem = FFNOptimizationProblem(model)

# Initialize the algorithm with the initial population
algorithm = NSGA2(pop_size=4, sampling=initial_population)

# Run the optimization
res = minimize(problem, algorithm, ('n_gen', 5), verbose=True)

print("Best solution found: %s" % res.F[0])
#print("Best variables found: %s" % res.X[0])

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        4 |      1 |             - |             -
     2 |        8 |      4 |  1.0000000000 |         ideal
     3 |       12 |      4 |  3.558937E+01 |         nadir
     4 |       16 |      1 |  0.1406791210 |         nadir
     5 |       20 |      1 |  0.0145530105 |         ideal
Best solution found: [0.41510743 0.42966044 0.         0.         0.        ]


In [98]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.8 MB/s eta 0:00:00


In [101]:
import optuna

def objective(trial):
    # Suggest hyperparameters
    hidden_sizes = [trial.suggest_int('n_units_l1', 5, 10),
                    trial.suggest_int('n_units_l2', 5, 10),
                    trial.suggest_int('n_units_l3', 5, 10),
                    trial.suggest_int('n_units_l4', 5, 10),
                    trial.suggest_int('n_units_l5', 5, 10)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Define the model
    model = FFN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=2)


    train_model(model, X_train_tensor, y_train_tensor)
    adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=learning_rate), 'cpu', epsilon=0.2, attack_type='fgsm')
    # Evaluation
    accuracy = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
tt = []
for key, value in trial.params.items():
    tt.append(value)
    print("    {}: {}".format(key, value))
lerr = tt[-1]
tt = tt[: -1]

[I 2025-02-14 21:10:23,271] A new study created in memory with name: no-name-446408de-b307-4ed3-b019-1bbabce82183
<ipython-input-101-95cca50a698a>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Epoch 1/10, Loss: 93.9818
Epoch 2/10, Loss: 33.2435
Epoch 3/10, Loss: 22.8494
Epoch 4/10, Loss: 20.8180
Epoch 5/10, Loss: 20.5991
Epoch 6/10, Loss: 20.3950
Epoch 7/10, Loss: 20.3620
Epoch 8/10, Loss: 20.4739
Epoch 9/10, Loss: 19.8244


[I 2025-02-14 21:10:41,309] Trial 0 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 6, 'n_units_l2': 6, 'n_units_l3': 7, 'n_units_l4': 9, 'n_units_l5': 9, 'learning_rate': 0.0003408637924613585}. Best is trial 0 with value: 0.429660439491272.


Epoch 10/10, Loss: 19.6798
Epoch 1/10, Loss: 0.4337
Epoch 2/10, Loss: 0.4341
Epoch 3/10, Loss: 0.4332
Epoch 4/10, Loss: 0.4330
Epoch 5/10, Loss: 0.4330
Epoch 6/10, Loss: 0.4326
Epoch 7/10, Loss: 0.4321
Epoch 8/10, Loss: 0.4320
Epoch 9/10, Loss: 0.4319
Epoch 10/10, Loss: 0.4317
Epoch 1/10, Loss: 70.2112
Epoch 2/10, Loss: 39.0958
Epoch 3/10, Loss: 28.6126
Epoch 4/10, Loss: 22.6022
Epoch 5/10, Loss: 20.7219
Epoch 6/10, Loss: 21.0531
Epoch 7/10, Loss: 20.0440
Epoch 8/10, Loss: 19.3930
Epoch 9/10, Loss: 20.1620
Epoch 10/10, Loss: 19.0815
Epoch 1/10, Loss: 0.4672
Epoch 2/10, Loss: 0.4737
Epoch 3/10, Loss: 0.4650
Epoch 4/10, Loss: 0.4564
Epoch 5/10, Loss: 0.4599
Epoch 6/10, Loss: 0.4595
Epoch 7/10, Loss: 0.4533
Epoch 8/10, Loss: 0.4506
Epoch 9/10, Loss: 0.4518


[I 2025-02-14 21:10:59,128] Trial 1 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 7, 'n_units_l2': 9, 'n_units_l3': 6, 'n_units_l4': 10, 'n_units_l5': 10, 'learning_rate': 0.0015491114483330314}. Best is trial 0 with value: 0.429660439491272.


...

Epoch 10/10, Loss: 19.3974
Epoch 1/10, Loss: 0.4416
Epoch 2/10, Loss: 2.9967
Epoch 3/10, Loss: 1.0840
Epoch 4/10, Loss: 0.5302
Epoch 5/10, Loss: 0.6388
Epoch 6/10, Loss: 0.6546
Epoch 7/10, Loss: 0.6618
Epoch 8/10, Loss: 0.6879
Epoch 9/10, Loss: 0.6579
Epoch 10/10, Loss: 0.6726
Epoch 1/10, Loss: 54.2163
Epoch 2/10, Loss: 28.5666
Epoch 3/10, Loss: 23.9529
Epoch 4/10, Loss: 23.5909
Epoch 5/10, Loss: 22.0792
Epoch 6/10, Loss: 20.7389
Epoch 7/10, Loss: 20.2782
Epoch 8/10, Loss: 20.7698
Epoch 9/10, Loss: 19.7028
Epoch 10/10, Loss: 19.5254
Epoch 1/10, Loss: 0.4377
Epoch 2/10, Loss: 0.4474
Epoch 3/10, Loss: 0.4354
Epoch 4/10, Loss: 0.4363
Epoch 5/10, Loss: 0.4389
Epoch 6/10, Loss: 0.4366
Epoch 7/10, Loss: 0.4325


[I 2025-02-14 21:25:46,237] Trial 48 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 10, 'n_units_l2': 10, 'n_units_l3': 6, 'n_units_l4': 9, 'n_units_l5': 8, 'learning_rate': 0.0009633572352242412}. Best is trial 19 with value: 0.9715869426727295.


Epoch 8/10, Loss: 0.4303
Epoch 9/10, Loss: 0.4310
Epoch 10/10, Loss: 0.4320
Epoch 1/10, Loss: 77.1523
Epoch 2/10, Loss: 24.8499
Epoch 3/10, Loss: 21.8129
Epoch 4/10, Loss: 21.3093
Epoch 5/10, Loss: 20.4874
Epoch 6/10, Loss: 20.8241
Epoch 7/10, Loss: 19.4492
Epoch 8/10, Loss: 19.4434
Epoch 9/10, Loss: 19.4718


[I 2025-02-14 21:26:04,119] Trial 49 finished with value: 0.570339560508728 and parameters: {'n_units_l1': 7, 'n_units_l2': 8, 'n_units_l3': 10, 'n_units_l4': 8, 'n_units_l5': 5, 'learning_rate': 0.06622416834202619}. Best is trial 19 with value: 0.9715869426727295.


Epoch 10/10, Loss: 19.1201
Epoch 1/10, Loss: 0.4012
Epoch 2/10, Loss: 8.9171
Epoch 3/10, Loss: 1.1330
Epoch 4/10, Loss: 0.6906
Epoch 5/10, Loss: 0.6888
Epoch 6/10, Loss: 0.6895
Epoch 7/10, Loss: 0.6903
Epoch 8/10, Loss: 0.6894
Epoch 9/10, Loss: 0.6893
Epoch 10/10, Loss: 0.6892
Best trial:
  Value: 0.9715869426727295
  Params: 
    n_units_l1: 9
    n_units_l2: 10
    n_units_l3: 5
    n_units_l4: 9
    n_units_l5: 8
    learning_rate: 0.05391227690499994
